In [1]:
#!pip install bs4
import bs4
import requests
import sys
import pandas as pd
import re
from matplotlib import pyplot as plt
import seaborn as sns
import json
import time
import ast
import random as ran
from scipy import stats
import numpy as np
from scipy.stats import t
from numpy.polynomial.polynomial import polyfit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

### Scraping Movies Data from IMDb Website using Beautiful Soup and Data Cleaning

In [2]:
"""

def extractcolumns(col,b):   
    data_d = {}
    name = col.find('a')
    year = col.find('span',{'class': 'lister-item-year'})
    rating = col.find('div',{'class':'inline-block ratings-imdb-rating'})
    m_score = col.find('span',{'class':'metascore'})
    votes = col.find('span',{'name':'nv'})
    gross = cols[b].findAll('span',{'name':'nv'})[-1]
    genre = cols[b].find('span', class_='genre')
    director = col.find_all('p')[2]    
    data_d['Movie_Name'] = name.get_text() if name else None
    if year.contents:
        data_d['Relese_Year'] = str(year.contents[0][1:-1].replace('I) (','')).strip() if year else None
    data_d['Rating_Score'] = float(rating.get('data-value')) if rating else None
    data_d['Meta_Score'] = float(m_score.contents[0].strip()) if m_score else None
    data_d['Total_Votes'] = int(votes.get('data-value').replace(",","")) if votes else None
    data_d['Gross_USA'] = int(gross.get('data-value').replace(",","")) if gross else None
    data_d['Movie_Genre'] = str(genre.get_text().replace('\n','').replace('"','').strip()) if genre else None
    data_d['Director_Star'] = director.text.replace('\n','').replace('    ','') if director else None    
    return data_d


#Provide the Imdb Website from which data needs to be scraped in 'movie_url' and movies count 
#required in 'target'. Years in the movie_url need to be changed according to the data

movie_dataset = [] 
movie_url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2016-01-01,2016-12-31&sort=boxoffice_gross_us,desc&start='
#Provide the 
target = 200
start = 0
end = 0 
count = 1 
tcount = target - end  
while tcount > 0:
    url = movie_url + str(count)
    source = requests.get(url).text
    soup = bs4.BeautifulSoup(source,'html.parser')       
    cols = soup.findAll('div',{'class':'lister-item-content'}) 
    movie_dataset.extend([extractcolumns(cols[b],b) for b in range(len(cols))])          
    start = int(soup.find("div", {"class":"nav"}).find("div", {"class": "desc"}).contents[1].get_text().split("-")[0].replace(",",""))
    end = int(soup.find("div", {"class":"nav"}).find("div", {"class": "desc"}).contents[1].get_text().split("-")[1].split(" ")[0].replace(",",  ""))
    tcount = target - end       
    count = end + 1       
e_data = pd.DataFrame(movie_dataset)
e_data.to_csv('MovieTest.csv')
print("Extraction Completed")

df_movies2014 = pd.read_csv('Movies2014.csv')
df_movies2015 = pd.read_csv('Movies2015.csv')
df_movies2016 = pd.read_csv('Movies2016.csv')
df_movies2017 = pd.read_csv('Movies2017.csv')
df_movies2018 = pd.read_csv('Movies2018.csv')
df_movies2019 = pd.read_csv('Movies2019.csv')
df_movies2020 = pd.read_csv('Movies2020.csv')
df_movies2021 = pd.read_csv('Movies2021.csv')

#Merging all the movie datasets of years 2014-2022 into one Merged_Movies Dataset 

files = ['Movies2014.csv','Movies2015.csv', 'Movies2016.csv',
         'Movies2017.csv', 'Movies2018.csv', 'Movies2019.csv', 'Movies2020.csv', 'Movies2021.csv']
df_list = []
for file in files:
    df = pd.read_csv(file)
    df_list.append(df)
merged_df = pd.concat(df_list)
merged_df.to_csv('Merged_Movies.csv', index=False)
df_merged_c = pd.read_csv('Merged_Movies.csv')
df_merged_c.drop(df_merged_c.filter(regex="Unname"),axis=1, inplace=True) 
df['Movie_Name'] = df['Movie_Name'].str.strip()
chinese_movies = df[df['Movie_Name'].str.contains('[\u4e00-\u9fff]')]
rows_to_drop = chinese_movies.index

df = df_merged_c

df = df.drop(rows_to_drop)
df = df.dropna(axis=0)
df[['Director', 'Stars']] = df['Director_Star'].str.split('|', expand = True)
df['Director'] = df['Director'].str.replace('Director:', '')
df['Director'] = df['Director'].str.replace('Directors:', '')
df['Director'] = df['Director'].str.replace('Stars:', '')
df['Stars'] = df['Stars'].str.replace('Stars:', '')
df.drop(df.filter(regex="Director_Star"),axis=1, inplace=True)
df['Gross_USA'] = df['Gross_USA'].astype('int64')
df = df.dropna()
df["Cast"] = df.apply(lambda row: row["Director"] + ", " + row["Stars"], axis=1)

df = df.drop(["Director", "Stars"], axis=1)
df = df.sort_values(by='Gross_USA', ascending=False)
df = df.reset_index()
df = df.drop("Unnamed: 0", axis=1)

"""

'\n\ndef extractcolumns(col,b):   \n    data_d = {}\n    name = col.find(\'a\')\n    year = col.find(\'span\',{\'class\': \'lister-item-year\'})\n    rating = col.find(\'div\',{\'class\':\'inline-block ratings-imdb-rating\'})\n    m_score = col.find(\'span\',{\'class\':\'metascore\'})\n    votes = col.find(\'span\',{\'name\':\'nv\'})\n    gross = cols[b].findAll(\'span\',{\'name\':\'nv\'})[-1]\n    genre = cols[b].find(\'span\', class_=\'genre\')\n    director = col.find_all(\'p\')[2]    \n    data_d[\'Movie_Name\'] = name.get_text() if name else None\n    if year.contents:\n        data_d[\'Relese_Year\'] = str(year.contents[0][1:-1].replace(\'I) (\',\'\')).strip() if year else None\n    data_d[\'Rating_Score\'] = float(rating.get(\'data-value\')) if rating else None\n    data_d[\'Meta_Score\'] = float(m_score.contents[0].strip()) if m_score else None\n    data_d[\'Total_Votes\'] = int(votes.get(\'data-value\').replace(",","")) if votes else None\n    data_d[\'Gross_USA\'] = int(gross

### Extracting Movie Overview, Keywords, Images using TMDB API

In [3]:
'''

sample_df1 = df.iloc[0:10000]
sample_df2 = df.iloc[10000:20000]
sample_df3 = df.iloc[20000:30000]
sample_df4 = df.iloc[30000:40000]
sample_df5 = df.iloc[40000:50000]

import requests
movie_data = {}

def makeRequest(query): 
    url = 'https://api.themoviedb.org/3/search/movie'
    api_key = 'api_key'
    # query = 'The Godfather'
    params = {'api_key': api_key, 'language': 'en-US', 'query': query, 'page': 1, 'include_adult': False}
    response = requests.get(url, params=params)
    movie = response.json()
    if(len(movie['results']) != 0):
        movie_data[query] = movie['results'][0]
    return [movie['results'][0]['id'] if(len(movie['results']) != 0) else 0, movie['results'][0]['overview'] if(len(movie['results']) != 0) else "Not Available"]


def makeRequestKeywords(movieId): 
    url = f"https://api.themoviedb.org/3/movie/%s/keywords" % (movieId)
    api_key = 'api_key'
    # query = 'The Godfather'
    params = {'api_key': api_key}
    response = requests.get(url, params=params)
    movie = response.json()
    return movie['keywords'] if 'keywords' in movie else []

def getData(name, index):
    print(index)
    movie_id, overview = makeRequest(name)
    keywords = makeRequestKeywords(movie_id)
    return [keywords, overview]
    
add_keywords_and_overview = lambda row: getData(row["Movie_Name"], row.name)
sample_df1[["keywords", "overview"]] = sample_df1.apply(add_keywords_and_overview, axis=1, result_type="expand")
sample_df2[["keywords", "overview"]] = sample_df2.apply(add_keywords_and_overview, axis=1, result_type="expand")
sample_df3[["keywords", "overview"]] = sample_df3.apply(add_keywords_and_overview, axis=1, result_type="expand")
sample_df4[["keywords", "overview"]] = sample_df4.apply(add_keywords_and_overview, axis=1, result_type="expand")
sample_df5[["keywords", "overview"]] = sample_df5.apply(add_keywords_and_overview, axis=1, result_type="expand")
merged_movies_new = pd.concat([sample_df1, sample_df2, sample_df3, sample_df4, sample_df5], axis=0)
merged_movies_new
merged_movies_new.to_csv("merged_movies_new.csv")

#Extracting Movie Image URL's to Json File to load the images in Website

merged_movie_data = {}
for i in range(1, 6):
    data = []
    with open('movie_data-' + str(i) + '.json', 'r') as file:
        data = json.load(file)
        merged_movie_data.update(data)
filename = "merged_movie_data.json"
with open(filename, "w") as json_file:
    # Dump the dictionary into the JSON file
    json.dump(merged_movie_data, json_file)
    
'''

'\n\nsample_df1 = df.iloc[0:10000]\nsample_df2 = df.iloc[10000:20000]\nsample_df3 = df.iloc[20000:30000]\nsample_df4 = df.iloc[30000:40000]\nsample_df5 = df.iloc[40000:50000]\n\nimport requests\nmovie_data = {}\n\ndef makeRequest(query): \n    url = \'https://api.themoviedb.org/3/search/movie\'\n    api_key = \'api_key\'\n    # query = \'The Godfather\'\n    params = {\'api_key\': api_key, \'language\': \'en-US\', \'query\': query, \'page\': 1, \'include_adult\': False}\n    response = requests.get(url, params=params)\n    movie = response.json()\n    if(len(movie[\'results\']) != 0):\n        movie_data[query] = movie[\'results\'][0]\n    return [movie[\'results\'][0][\'id\'] if(len(movie[\'results\']) != 0) else 0, movie[\'results\'][0][\'overview\'] if(len(movie[\'results\']) != 0) else "Not Available"]\n\n\ndef makeRequestKeywords(movieId): \n    url = f"https://api.themoviedb.org/3/movie/%s/keywords" % (movieId)\n    api_key = \'api_key\'\n    # query = \'The Godfather\'\n    para

In [4]:
def extract_names(keywords):
    if pd.isnull(keywords): 
        return "Not Available"
    keywords = ast.literal_eval(keywords)
    names = []
    for keyword in keywords:
        names.append(keyword['name'])       
    return ','.join(names) if len(names) != 0 else "Not Available"

merged_df = pd.read_csv('final_merged_movies.csv')
merged_df = merged_df[["Movie_Name", "Rating_Score", "Movie_Genre", "Cast","Gross_USA", "keywords", "overview", "Total_Votes"]]
merged_df

,Movie_Name,Rating_Score,Movie_Genre,Cast,Gross_USA,keywords,overview,Total_Votes
0,Star Wars: Episode VII - The Force Awakens,7.8,"Action, Adventure, Sci-Fi","J.J. Abrams, Daisy Ridley, John Boyega, Oscar...",936662225.0,"android,spacecraft,space opera",Thirty years after defeating the Galactic Empi...,937561.0
1,Avengers: Endgame,8.4,"Action, Adventure, Drama","Anthony Russo, Joe Russo, Robert Downey Jr., ...",858373000.0,"space travel,time travel,time machine,sequel,s...",After the devastating events of Avengers: Infi...,1147483.0
2,Spider-Man: No Way Home,8.2,"Action, Adventure, Fantasy","Jon Watts, Tom Holland, Zendaya, Benedict Cum...",804747988.0,"new york city,loss of loved one,showdown,secre...",Peter Parker is unmasked and no longer able to...,770024.0
3,Jurassic World,6.9,"Action, Adventure, Sci-Fi","Colin Trevorrow, Chris Pratt, Bryce Dallas Ho...",652270625.0,"dna,tyrannosaurus rex,velociraptor,island,anim...",Twenty-two years after the events of Jurassic ...,649199.0
4,Star Wars: Episode VIII - The Last Jedi,6.9,"Action, Adventure, Fantasy","Rian Johnson, Daisy Ridley, John Boyega, Mark...",620181382.0,"bunker,space battle,failure,defeat,sequel,spac...",Rey develops her newly discovered abilities wi...,638768.0
...,...,...,...,...,...,...,...,...
18905,Death in Berruecos,7.1,"History, Thriller","Caupolican Ovalles, Luis Gerónimo Abreu, Laur...",8.0,historical,Ten years after General Antonio José de Sucre ...,8.0
18906,Hells Bells,5.8,Western,"John Bock, Bibiana Beglau, Lars Eidinger, Fra...",8.0,"hell,cartoon,bell,demon,short film","The demons of hell play music for Satan, whose...",8.0
18907,Ayuda Babes,5.8,Comedy,"Joven Tan, Gardo Versoza, Ate Gay, Joey Paras...",8.0,"lgbt,covid-19",A hilarious look at the altered lifestyles of ...,8.0
18908,Victim,5.3,Drama,"Faizal Khan, Kuldeep Singh, Mohit Nain, Saafi...",8.0,"london, england,blackmail,controversy,film noi...",Barrister Melville Farr is on the path to succ...,8.0


In [5]:
merged_df['Gross_USA'] = merged_df['Gross_USA'].astype(float)
merged_df = merged_df.replace('', np.nan).dropna()
merged_df = merged_df.sort_values(by='Gross_USA', ascending=False)
display(merged_df.head(10))

,Movie_Name,Rating_Score,Movie_Genre,Cast,Gross_USA,keywords,overview,Total_Votes
0,Star Wars: Episode VII - The Force Awakens,7.8,"Action, Adventure, Sci-Fi","J.J. Abrams, Daisy Ridley, John Boyega, Oscar...",936662225.0,"android,spacecraft,space opera",Thirty years after defeating the Galactic Empi...,937561.0
1,Avengers: Endgame,8.4,"Action, Adventure, Drama","Anthony Russo, Joe Russo, Robert Downey Jr., ...",858373000.0,"space travel,time travel,time machine,sequel,s...",After the devastating events of Avengers: Infi...,1147483.0
2,Spider-Man: No Way Home,8.2,"Action, Adventure, Fantasy","Jon Watts, Tom Holland, Zendaya, Benedict Cum...",804747988.0,"new york city,loss of loved one,showdown,secre...",Peter Parker is unmasked and no longer able to...,770024.0
3,Jurassic World,6.9,"Action, Adventure, Sci-Fi","Colin Trevorrow, Chris Pratt, Bryce Dallas Ho...",652270625.0,"dna,tyrannosaurus rex,velociraptor,island,anim...",Twenty-two years after the events of Jurassic ...,649199.0
4,Star Wars: Episode VIII - The Last Jedi,6.9,"Action, Adventure, Fantasy","Rian Johnson, Daisy Ridley, John Boyega, Mark...",620181382.0,"bunker,space battle,failure,defeat,sequel,spac...",Rey develops her newly discovered abilities wi...,638768.0
5,The Lion King,6.8,"Animation, Adventure, Drama","Jon Favreau, Donald Glover, Beyoncé, Seth Rog...",543638043.0,"loss of loved one,lion,father murder,manipulat...",A young lion prince is cast out of his pride b...,249159.0
6,Rogue One: A Star Wars Story,7.8,"Action, Adventure, Sci-Fi","Gareth Edwards, Felicity Jones, Diego Luna, A...",532177324.0,"rebel,spacecraft,space battle,space travel,pre...",A rogue band of resistance fighters unite for ...,647094.0
7,Star Wars: The Rise Of Skywalker,6.5,"Action, Adventure, Fantasy","J.J. Abrams, Daisy Ridley, John Boyega, Oscar...",515202542.0,space opera,The surviving Resistance faces the First Order...,457952.0
8,Beauty and the Beast,7.1,"Adventure, Family, Fantasy","Bill Condon, Emma Watson, Dan Stevens, Luke E...",504014165.0,"prince,castle,rose,musical,princess,beast,la b...","Follow the adventures of Belle, a bright young...",21517.0
9,Finding Dory,7.3,"Animation, Adventure, Comedy","Andrew Stanton, Angus MacLane, Ellen DeGenere...",486295561.0,"fish,amnesia,octopus,sequel,anthropomorphism,u...",Dory is reunited with her friends Nemo and Mar...,281491.0


In [6]:
#Weighted Average Calculation
merged_df['Rating_Score'] = merged_df['Rating_Score'].astype(float)
merged_df['Total_Votes'] = merged_df['Total_Votes'].astype(float)
merged_df['weighted_rating'] = ((merged_df['Rating_Score'] * merged_df['Total_Votes']) / (merged_df['Total_Votes'].sum()))
merged_df.head()

,Movie_Name,Rating_Score,Movie_Genre,Cast,Gross_USA,keywords,overview,Total_Votes,weighted_rating
0,Star Wars: Episode VII - The Force Awakens,7.8,"Action, Adventure, Sci-Fi","J.J. Abrams, Daisy Ridley, John Boyega, Oscar...",936662225.0,"android,spacecraft,space opera",Thirty years after defeating the Galactic Empi...,937561.0,0.039321
1,Avengers: Endgame,8.4,"Action, Adventure, Drama","Anthony Russo, Joe Russo, Robert Downey Jr., ...",858373000.0,"space travel,time travel,time machine,sequel,s...",After the devastating events of Avengers: Infi...,1147483.0,0.051827
2,Spider-Man: No Way Home,8.2,"Action, Adventure, Fantasy","Jon Watts, Tom Holland, Zendaya, Benedict Cum...",804747988.0,"new york city,loss of loved one,showdown,secre...",Peter Parker is unmasked and no longer able to...,770024.0,0.033951
3,Jurassic World,6.9,"Action, Adventure, Sci-Fi","Colin Trevorrow, Chris Pratt, Bryce Dallas Ho...",652270625.0,"dna,tyrannosaurus rex,velociraptor,island,anim...",Twenty-two years after the events of Jurassic ...,649199.0,0.024086
4,Star Wars: Episode VIII - The Last Jedi,6.9,"Action, Adventure, Fantasy","Rian Johnson, Daisy Ridley, John Boyega, Mark...",620181382.0,"bunker,space battle,failure,defeat,sequel,spac...",Rey develops her newly discovered abilities wi...,638768.0,0.023699


### Content-Based Movie Recommendation System  (TF-IDF, Cosine Similarity)

In [7]:
movies_df = merged_df
movies_df['Cast'] = movies_df['Cast'].apply(lambda x: ' '.join(x.split(',')[:20]))
movies_df['Movie_Genre'] = movies_df['Movie_Genre'].str.replace(',', ' ') # Replace commas with spaces
tfidf = TfidfVectorizer(stop_words='english')
movies_df['Movie_Genre'] = movies_df['Movie_Genre'].fillna('')
tfidf_matrix = tfidf.fit_transform(movies_df['Movie_Name'] + ' ' + movies_df['Movie_Genre'] + ' ' + movies_df['Cast'] + ' ' + movies_df['overview'] + ' ' + movies_df['keywords'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
def get_recommendations(title, cosine_sim=cosine_sim):
    indices = pd.Series(movies_df.index, index=movies_df['Movie_Name']).drop_duplicates()
    idx = int(indices[title])
    sim_scores = list(enumerate(cosine_sim[idx]))
    rating_scores = [float(score) for score in movies_df['Rating_Score']]
    weighted_scores = [(i, sim_scores[i][1] * rating_scores[i]) for i in range(len(sim_scores))]
    weighted_scores = sorted(weighted_scores, key=lambda x: x[1], reverse=True)
    movie_indices = [i[0] for i in weighted_scores[1:20]]
    return movies_df[['Movie_Name', 'weighted_rating']].iloc[movie_indices].sort_values(by='weighted_rating', ascending=False)

In [8]:
get_recommendations('Avengers: Age of Ultron')

,Movie_Name,weighted_rating
1,Avengers: Endgame,5.182734e-02
31,Guardians of the Galaxy,5.117230e-02
40,Captain America: The Winter Soldier,3.581526e-02
2,Spider-Man: No Way Home,3.395092e-02
15,Captain America: Civil War,3.342061e-02
36,Thor: Ragnarok,3.234720e-02
48,Doctor Strange,3.031622e-02
18,Guardians of the Galaxy Vol. 2,2.821944e-02
72,Ant-Man,2.650053e-02
30,Spider-Man: Homecoming,2.649964e-02


### Content-Based Movie Recommendation System  (Count Vectorizer, Cosine Similarity)

In [9]:
count_vec = CountVectorizer(stop_words='english')
movies_df['Movie_Genre'] = movies_df['Movie_Genre'].fillna('')
count_matrix = tfidf.fit_transform(movies_df['Movie_Name'] + ' ' + movies_df['Movie_Genre'] + ' ' + movies_df['Cast'] + ' ' + movies_df['overview'] + ' ' + movies_df['keywords'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

def get_recommendations(title, cosine_sim=cosine_sim):
    indices = pd.Series(movies_df.index, index=movies_df['Movie_Name']).drop_duplicates()
    idx = int(indices[title])
    sim_scores = list(enumerate(cosine_sim[idx]))
    rating_scores = [float(score) for score in movies_df['Rating_Score']]
    weighted_scores = [(i, sim_scores[i][1] * rating_scores[i]) for i in range(len(sim_scores))]
    weighted_scores = sorted(weighted_scores, key=lambda x: x[1], reverse=True)
    movie_indices = [i[0] for i in weighted_scores[1:21]]
    return movies_df[['Movie_Name', 'weighted_rating']].iloc[movie_indices].sort_values(by='weighted_rating', ascending=False)

In [10]:
get_recommendations('Avengers: Age of Ultron')

,Movie_Name,weighted_rating
1,Avengers: Endgame,5.182734e-02
31,Guardians of the Galaxy,5.117230e-02
40,Captain America: The Winter Soldier,3.581526e-02
2,Spider-Man: No Way Home,3.395092e-02
15,Captain America: Civil War,3.342061e-02
36,Thor: Ragnarok,3.234720e-02
48,Doctor Strange,3.031622e-02
18,Guardians of the Galaxy Vol. 2,2.821944e-02
72,Ant-Man,2.650053e-02
30,Spider-Man: Homecoming,2.649964e-02


### Content-Based Movie Recommendation System using the K-Nearest Neighbors Algorithm (TF-IDF)

In [11]:
tfidf = TfidfVectorizer(stop_words='english')
movies_df['Movie_Genre'] = movies_df['Movie_Genre'].fillna('')
tfidf_matrix = tfidf.fit_transform(movies_df['Movie_Name'] + ' ' + movies_df['Movie_Genre'] + ' ' + movies_df['Cast'] + ' ' + movies_df['overview'] + ' ' + movies_df['keywords'])
knn = NearestNeighbors(n_neighbors=21, algorithm='brute', metric='cosine')
knn.fit(tfidf_matrix)
def get_recommendations(title, knn_model=knn, tfidf_matrix=tfidf_matrix):
    indices = pd.Series(movies_df.index, index=movies_df['Movie_Name']).drop_duplicates()
    idx = int(indices[title])
    distances, indices = knn_model.kneighbors(tfidf_matrix[idx])
    movie_indices = indices[0][1:21]
    return movies_df[['Movie_Name', 'weighted_rating']].iloc[movie_indices].sort_values(by='weighted_rating', ascending=False)

In [12]:
get_recommendations('Avengers: Age of Ultron')

,Movie_Name,weighted_rating
1,Avengers: Endgame,5.182734e-02
31,Guardians of the Galaxy,5.117230e-02
40,Captain America: The Winter Soldier,3.581526e-02
15,Captain America: Civil War,3.342061e-02
36,Thor: Ragnarok,3.234720e-02
48,Doctor Strange,3.031622e-02
18,Guardians of the Galaxy Vol. 2,2.821944e-02
72,Ant-Man,2.650053e-02
30,Spider-Man: Homecoming,2.649964e-02
13,Captain Marvel,2.075662e-02


### Content-Based Movie Recommendation System using the K-Nearest Neighbors Algorithm (Count Vectorizer)

In [13]:
count = CountVectorizer(stop_words='english')
movies_df['Movie_Genre'] = movies_df['Movie_Genre'].fillna('')
count_matrix = count.fit_transform(movies_df['Movie_Name'] + ' ' + movies_df['Movie_Genre'] + ' ' + movies_df['Cast'] + ' ' + movies_df['overview'] + ' ' + movies_df['keywords'])
def get_recommendations(title, knn_model=knn, count_matrix=count_matrix):
    indices = pd.Series(movies_df.index, index=movies_df['Movie_Name']).drop_duplicates()
    idx = int(indices[title])
    distances, indices = knn_model.kneighbors(count_matrix[idx])
    movie_indices = indices[0][1:21]
    return movies_df[['Movie_Name', 'weighted_rating']].iloc[movie_indices].sort_values(by='weighted_rating', ascending=False)

In [14]:
get_recommendations('Avengers: Age of Ultron')

,Movie_Name,weighted_rating
1,Avengers: Endgame,5.182734e-02
31,Guardians of the Galaxy,5.117230e-02
40,Captain America: The Winter Soldier,3.581526e-02
15,Captain America: Civil War,3.342061e-02
36,Thor: Ragnarok,3.234720e-02
48,Doctor Strange,3.031622e-02
18,Guardians of the Galaxy Vol. 2,2.821944e-02
72,Ant-Man,2.650053e-02
30,Spider-Man: Homecoming,2.649964e-02
13,Captain Marvel,2.075662e-02
